X.shape:(625, 6)
z.shape: (25, 25)


In [34]:
from typing import Callable

def cross_val(k:int, model:str, lmb:int, X:np.ndarray, z:np.ndarray)-> np.ndarray:    
    if model == "ridge":  
        model = RidgeRegression(lambda_val = lmb)
    if model == "lasso":
        model = LassoRegression(alpha = lmb)
    if model == "OLS":
        model = OLS()
    
    kfold = KFold(n_splits = k)
    scores_KFold = np.zeros(k)
    z = z.ravel()
    #scores_KFold idx counter         
    j = 0
    for train_inds, test_inds in kfold.split(X,z):
        # print(f"train_inds: {train_inds}")
        # print(f"test_inds: {test_inds}")
        
        #get all cols and selected train_inds rows:
        xtrain = X[train_inds,:]
        ytrain = z[train_inds]
        #get all cols and selected test_inds rows:
        xtest = X[test_inds,:]
        ytest = z[test_inds]
        model.fit(xtrain,ytrain)
        ypred = model.predict(xtest)
        scores_KFold[j] = np.sum((ypred - ytest)**2)/np.size(ypred)
        j += 1

    return np.mean(scores_KFold)

In [30]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from common import *

# A seed just to ensure that the random numbers are the same for every run.
# Useful for eventual debugging.
np.random.seed(4155)

def FrankeFunction(x: float ,y: float) -> float:
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

def create_X(x:np.ndarray, y:np.ndarray, n:int)->np.ndarray:
    if (len(x.shape)) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    l = int((n+1)*(n+2)/2) # Number of elements in beta
    X = np.ones((N,l))

    for i in range(1,n+1):
        q = int((i)*(i+1)/2)
        for k in range(i+1):
            X[:,q+k] = (x**(i-k))*(y**k)

    return X

# Generate the data.
n = 25
x = np.sort(np.random.uniform(0, 1, n))
y = np.sort(np.random.uniform(0, 1, n))
x,y = np.meshgrid(x,y)

degree = 2
X = create_X(x,y,degree)
print(f"X.shape:{X.shape}")
z = FrankeFunction(x, y) + 0.3*np.random.normal(0, size = n)
print(f"z.shape: {z.shape}")
# Decide degree on polynomial to fit

# Decide which values of lambda to use
nlambdas = 10
lambdas = np.logspace(-3, 5, nlambdas)


625